In [ ]:
import numpy as np
import ctypes

# Load GSL library
gsl = ctypes.CDLL("libgsl.so")

# Define the matrices
matrices = [
    np.array([[3, -1, 1],
              [3, 6, 2],
              [3, 3, 7]], dtype=np.double),
    np.array([[10, -1, 0],
              [-1, 10, -2],
              [0, -2, 10]], dtype=np.double),
    np.array([[10, 5, 0, 0],
              [5, 10, -4, 0],
              [0, -4, 8, -1],
              [0, 0, -1, 5]], dtype=np.double),
    np.array([[4, 1, 1, 0, 1],
              [-1, -3, 1, 1, 0],
              [2, 1, 5, -1, -1],
              [-1, -1, -1, 4, 0],
              [0, 2, -1, 1, 4]], dtype=np.double)
]

# Define the LU decomposition function using GSL
def lu_decomposition(matrix):
    rows, cols = matrix.shape
    size = rows * cols
    data = matrix.flatten()

    # Allocate memory for matrix and permutation vector
    gsl_matrix = (ctypes.c_double * size)(*data)
    p = (ctypes.c_size_t * size)()

    # Perform LU decomposition
    gsl_result = gsl.gsl_linalg_LU_decomp(gsl_matrix, p, ctypes.byref(ctypes.c_int(rows)))

    # Extract L and U matrices from LU decomposition
    lu = np.array(gsl_matrix[:size]).reshape(rows, cols)
    permutation = np.array(p[:rows], dtype=np.int)

    return lu, permutation

# Verify correctness of LU decomposition
for i, matrix in enumerate(matrices):
    print(f"Matrix {i + 1}:")
    lu, permutation = lu_decomposition(matrix)

    # Reconstruct the original matrix using LU decomposition
    p_matrix = np.eye(matrix.shape[0])[permutation]
    l = np.tril(lu, k=-1) + np.eye(matrix.shape[0])
    u = np.triu(lu)
    reconstructed_matrix = np.dot(p_matrix.T, np.dot(l, u))

    # Compare original matrix with reconstructed matrix
    if np.allclose(matrix, reconstructed_matrix):
        print("LU decomposition is correct.")
    else:
        print("LU decomposition is incorrect.")
